In [2]:
from numpy import linspace, concatenate, sqrt
from scipy.stats import norm                        # lib. for statistical functions
from ipywidgets import interactive, FloatSlider, IntSlider    # lib. for interactive graphic
from bokeh.io import push_notebook, show, output_notebook # lib. for graphic output
from bokeh.plotting import figure
output_notebook()
options = dict(plot_height=250,plot_width=700,
               tools="pan,wheel_zoom,reset,save,crosshair,box_select")
def printred(s):
    display(HTML('<pre style="font: bold 12pt Courier, serif;color:#a02">' + s + '</pre>') )
def printgrn(s):
    display(HTML('<pre style="font: bold 12pt Courier, serif;color:#080">' + s + '</pre>') )
    
# this loads some HTML style files
from IPython.core.display import HTML
with open( './style/custom.css', 'r' ) as f: html_style = f.read()
HTML( html_style )

Loading BokehJS ...

# Lo Z-test (esempio)

Si sospetta che una certa terapia faccia aumentare la pressione diastolica. Nella popolazione generale la pressione diastolica ha distribuzione $N(\mu_0,\sigma^2)$ con $\mu_0=75$ e $\sigma=9.5$. 

Assumiamo che tra i pazienti in terapia la pressione diastolica sia distribuita normalmente con media ignota $\mu$ e con **la stessa deviazione standard** della popolazione generale. Vogliamo testare le seguenti ipotesi:


$\qquad H_0:\quad \mu=\mu_0$

$\qquad H_A:\quad \mu>\mu_0$

Il test consiste nel misurare la pressione ad un campione di $n$ pazienti e di questi dati calcolare la media. Abbiamo quindi la seguente statistica

$\displaystyle\qquad \bar X\ =\ \frac1n\sum_{i=1}^nX_i$

Dove $X_i$ è la v.a. che dà la pressione dell'$i$-esimo paziente del campione. Rigetteremo $H_0$ se il
valore ottenuto è suporiore ad un certo $x_\alpha$ che vogliamo fissare in modo che l'errore I tipo risulti uguale ad $\alpha$. Quindi $x_\alpha$ dev'essere tale che $x_\alpha$ tale che $\Pr(\bar X>x_\alpha)=\alpha$.

Se $H_0$ è vera, $\bar X\sim N\bigg(\mu_0,\dfrac{\sigma^2}{n}\bigg)$.

Se $H_A$ è vera, $\bar X\sim N\bigg(\mu,\dfrac{\sigma^2}{n}\bigg)$ per qualche $\mu>\mu_0$.

In [3]:
n = 10;  mu0 = 75;  sigma=9.5;  xmin= 65.; xmax = 85.; x =linspace(xmin,xmax,200)

def shade(xalpha,f):
    ab  = concatenate( ([xalpha], linspace(xalpha,xmax), [xmax]) )
    fab = concatenate( ([0], f(linspace(xalpha,xmax) ), [0] ) )
    return ab, fab

plot = figure(title="PDF di X ~ N( μ, (9.5)^2 / n )",
              x_axis_label = "pressione diastolica", 
              y_axis_label = "densità di probabilità", 
              x_range=(xmin,xmax), 
              y_range=(0,0.3), 
              **options )               
plot.title.text_font="courier"
plot.title.text_font_size="14pt"


xalpha = 80
px,py = shade(xalpha, lambda x: norm.pdf( x, mu0, sigma/sqrt(n) ) )
p1 = plot.patch( px, py, color='#991111', alpha=0.5 )
p2 = plot.line(x,  norm.pdf(x, mu0, sigma), color='black' )
# px,py = shade(xmax, lambda x: norm.pdf( x, mu0, sigma) )
# p3 = plot.patch( px, py, color='#991111', alpha=0.5 )
def f(xalpha, mu, n):
    px,py = shade(xalpha, lambda x: norm.pdf( x, mu, sigma/sqrt(n)) )
    p1.data_source.data['x'] = px
    p1.data_source.data['y'] = py  
    p2.data_source.data['y'] = norm.pdf(x,  mu, sigma/sqrt(n) )
    printred('T = [ x_α, +∞ )  zona di rifiuto  x_α = {}'.format(xalpha) )
    if mu == mu0: 
        printred('Pr( T | μ = {:.2f}) = α = {:.3f} (falso positivo) ERRORE I tipo'.format(mu,1-norm.cdf(xalpha,mu,sigma/sqrt(n) ) ) ) 
    else: 
        printgrn('Pr( T | μ = {:.2f}) = β  = {:.3f} (falso negtivo) ERRORE II tipo'.format(mu,norm.cdf(xalpha, mu, sigma/sqrt(n))))
    push_notebook()
    
show(plot, notebook_handle=True) 

interactive(f,
      xalpha = FloatSlider(description="x_α", min=mu0, max=80, step=0.2, value=80), 
      mu     = FloatSlider(description="μ", min=mu0, max=mu0+10, step=1, value=mu0),
      n      = IntSlider(description="n", min=n, max=n+40, step=5, value=n) )


interactive(children=(FloatSlider(value=80.0, description='x_α', max=80.0, min=75.0, step=0.2), FloatSlider(va…

# Domanda (1)

Con un capione di dimensione $25$ se vogliamo un una significatività del $5\%$ e una potenza del $95\%$, quanto è l'effect size $\delta $ che possiamo misurare?

# Risposta

Sia $\bar X$ la v.a. che da a media campionaria. Quindi $\bar X\sim N(\mu,\sigma^2/n)$.

$\Pr(\bar X>x_{5\%}\,|\, \mu=75) = 0.05$


$\Pr(\bar X>x_{5\%}\,|\, \mu=75+\delta) = 0.95$


Riduciamoci alla distribuzione normale standardizzata (N.B. $\sigma/\sqrt{n}= 1.9$)


$(1)\qquad \Pr\bigg( Z\le\dfrac{x_{5\%}-75}{1.9}\bigg) = 0.95$

$(2)\qquad \Pr\bigg( Z\le\dfrac{x_{5\%}-75-\delta}{1.9}\bigg) = 0.05$

Da (1) otteniamo

$\dfrac{x_{5\%}-75}{1.9}\ =\ $ `norm.ppf(0.95)`

In [3]:
norm.ppf(0.95)

1.6448536269514722

Quindi $x_{5\%}\ \cong\ 75 + 1.9 \cdot 1.64\ \cong\ 78.1$

Da (2) otteniamo

$\dfrac{x_{5\%}-75-\delta}{1.9}\ =\  $ `norm.ppf(0.05)` = `- norm.ppf(0.95)`

Quindi $\delta\ \cong\ 78.1 -75 +  1.9\cdot 1.64\ \cong\ 6.2$

# Domanda (2)

Dalla v.a. $\bar X$ qui soprta otteniamo la misura $\bar x = 79$. Qual è il p-valore associato al valore di questa statistica? Posso rifiutare l'ipotesi nulla con una sigificatià del $1\%\;$ ?<br><br>

# Risposta

$\Pr(\bar X>79\,|\, \mu=75)
\ =\ \Pr(Z>4/1.9)\ =\ $ p-valore

$\phantom{\Pr(\bar X>79\,|\, \mu=75)}
\ =\ \Pr(Z>4/1.9)\ =\ $ `1- norm.cdf(4/1.9)`

Il p-valore è > $1\%$ quindi non possi rigettare l'ipotesi nulla con la significatività richiesta.

In [4]:
1 - norm.cdf(4/1.9)

0.017634203613372312